<a href="https://colab.research.google.com/github/Rtrey29/DS-Unit-2-Tree-Ensembles/blob/master/module1-decision-trees/U2S7M1%20decision_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U pandas-profiling

     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 614kB 46.0MB/s 
     |████████████████████████████████| 225kB 48.5MB/s 
     |████████████████████████████████| 768kB 50.2MB/s 
     |████████████████████████████████| 51kB 23.8MB/s 
     |████████████████████████████████| 194kB 48.4MB/s 
     |████████████████████████████████| 51kB 23.8MB/s 
     |████████████████████████████████| 737kB 40.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/45/10/b5/5565b02c204a3bb87490a22e0e4c382ea06337bb79fd74d58a
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Stored in directory: /root/.cache/pip/wheels/b0/b2/96/2074eee7dbf7b7df69d004c9b6ac4e32dad04fb7666cf943bd
Successfully built pandas-profiling htmlmin confuse
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1

In [0]:
!pip install category_encoders

In [0]:
%matplotlib inline
import category_encoders as ce
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler

WEB = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/tanzania/'

train = pd.merge(pd.read_csv(WEB + 'train_features.csv'), 
                 pd.read_csv(WEB + 'train_labels.csv'))
test = pd.read_csv(WEB + 'test_features.csv')
sample_submission = pd.read_csv(WEB + 'sample_submission.csv')

# Split train into train & val
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)


train.shape, val.shape, test.shape

((47520, 41), (11880, 41), (14358, 40))

In [0]:
train.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,47520.000000,47520.000000,47520.000000,47520.000000,4.752000e+04,47520.000000,47520.000000,47520.000000,47520.000000,47520.000000
mean,37037.915699,321.925261,669.567656,34.082431,-5.705946e+00,0.477736,15.258291,5.616751,179.805513,1302.768939
std,21412.099719,3197.240487,693.005745,6.552053,2.941332e+00,13.312977,17.530228,9.621230,463.081564,950.955437
min,0.000000,0.000000,-63.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18482.750000,0.000000,0.000000,33.092142,-8.528215e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,36986.500000,0.000000,372.500000,34.910740,-5.021436e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55450.250000,25.000000,1320.000000,37.170578,-3.327185e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.344301,-2.000000e-08,1776.000000,99.000000,80.000000,15300.000000,2013.000000


In [0]:
train.describe(exclude='number')

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
count,47520,44616,44603,47520,47520,47234,47520,47520,47520,44876,47520,44392,24988,45077,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
unique,349,1716,1929,30661,9,17231,21,124,2082,2,1,12,2563,2,18,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6,3
top,2011-03-17,Government Of Tanzania,DWE,none,Lake Victoria,Shuleni,Iringa,Njombe,Igosi,True,GeoData Consultants Ltd,VWC,K,True,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
freq,474,7321,13978,2879,8137,420,4250,2003,257,40838,47520,29470,548,31071,21448,21448,21448,32449,42027,20287,20287,40598,40598,26567,26567,13620,13620,36638,22778,27642,25807


In [0]:
def wrangle(X):
  X = X.copy()
#   small coordinate values to 0
  X['latitude'] = X['latitude'].replace(-2e-08, 0)
  
#   replacing zeroes with mean values
  cols_with_zeroes = ['construction_year', 'longitude', 'latitude']
  for col in cols_with_zeroes:
    X[col] = X[col].replace(0, np.nan)
    X[col] = X[col].fillna(X[col].mean())
    
#     date recorded to date time format
  X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
  
#   extract year from datetime
  X['year_recorded'] = X['date_recorded'].dt.year
  
# dropping features
  X = X.drop(columns= ['quantity_group', 'payment_type'])
  
#   for categoricals with missing values fill with 'missing'
  categoricals = X.select_dtypes(exclude='number').columns
  for col in categoricals:
    X[col] = X[col].fillna('MISSING')
    
  return X

train = wrangle(train)
val = wrangle(val)
test= wrangle(test)


In [0]:
# define the target
target = 'status_group'

In [0]:
# get a DF with all train cols except target and id
train_features = train.drop(columns=[target, 'id'])

In [0]:
# get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

In [0]:
# get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

In [0]:
# geta list of all categorical features with cardinality <=50
categorical_features = cardinality[cardinality <=50].index.tolist()

In [0]:
# combine the lists
features = numeric_features + categorical_features

In [0]:
# Arrange data into Xfeatures matrix and Y vector
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]


In [0]:
# encoder fit_transform on train, transform on val and test
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)

# Standard scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

# declare the model, fit the model on train, score the val, predict on test
model = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)
model.fit(X_train_scaled, y_train)

print('Validation Accuracy', model.score(X_val_scaled, y_val))

y_pred = model.predict(X_test_scaled)

Validation Accuracy 0.7359427609427609


In [0]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)

# encoder fit_transform on train, transform on val and test
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)

# Standard scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)
X_test_scaled = scaler.transform(X_test_encoded)


model = dt
model.fit(X_train_scaled, y_train)

print('Validation Accuracy', model.score(X_val_scaled, y_val))

y_pred = model.predict(X_test_scaled)

Validation Accuracy 0.760016835016835


In [0]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=1000, min_samples_split=6,
                              criterion='gini', max_features='auto',
                              oob_score=True, random_state=42, n_jobs=-1)

# encoder fit_transform on train, transform on val and test
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)

# Standard scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)
X_test_scaled = scaler.transform(X_test_encoded)


model = rfc
model.fit(X_train_scaled, y_train)

print('Validation Accuracy', model.score(X_val_scaled, y_val))

y_pred = model.predict(X_test_scaled)

Validation Accuracy 0.8095959595959596


In [0]:
y_pred = model.predict(X_test_scaled)
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission-01.csv', index=False)

In [0]:
from google.colab import files
files.download('submission-01.csv')


In [0]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=1000, min_samples_split=6,
                              criterion='gini', max_features='auto',
                              oob_score=True, random_state=1, n_jobs=-1)

# encoder fit_transform on train, transform on val and test
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)

# Standard scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)
X_test_scaled = scaler.transform(X_test_encoded)


model = rfc
model.fit(X_train_scaled, y_train)

print('Validation Accuracy', model.score(X_val_scaled, y_val))

y_pred = model.predict(X_test_scaled)

Validation Accuracy 0.810942760942761
